#Prediksi Pendapatan Penduduk Menggunakan Metode Gaussian Naive Bayes Classifier

Bernadus Dwii Adicitra_215314052

Eustachia Agnesti Marta Da Silva_215314085

Lusia Juliana Silaban_215314087


In [ ]:
# Import libraries
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # for data visualization purposes
import seaborn as sns # for statistical data visualization
%matplotlib inline
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import dataset
data = '/content/drive/MyDrive/Colab Notebooks/Examples/adult.csv'

df = pd.read_csv(data, header=None, sep=',\s')

#  Step 1 - Eksploratory Data Analysis (EDA)

In [ ]:
# View dimensions of dataset
df.shape

In [ ]:
# View top 5 rows of dataset
df.head()

#Rename column names
We can see that the dataset does not have proper column names. The columns are merely labelled as 0,1,2.... and so on. We should give proper names to the columns. I will do it as follows:-

In [ ]:
col_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
             'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

df.columns = col_names

# df.columns

In [ ]:
# let's again preview the dataset
df.head()

In [ ]:
# view summary of dataset

df.info()

# a) Get descriptive atatistics for numerical and categorical features

In [ ]:
# Get descriptive statistics for numerical and categorical features.
print(df.describe(include='all'))

# b) Chek categorical of variables

In [ ]:
# find categorical variables

categorical = [var for var in df.columns if df[var].dtype=='O']

print('There are {} categorical variables\n'.format(len(categorical)))

print('The categorical variables are :\n\n', categorical)

In [ ]:
# view the categorical variables

df[categorical].head()

In [ ]:
# check missing values in categorical variables

df[categorical].isnull().sum()

We can see that there are no missing values in the categorical variables. I will confirm this further.

In [ ]:
# view frequency counts of values in categorical variables

for var in categorical:

    print(df[var].value_counts())

# c) replace '?' values in occupation variable with `NaN`

Now, we can see that there are no values encoded as ? in the workclass variable.

I will adopt similar approach with occupation and native_country column.

In [ ]:
# replace '?' values in 'df' with `NaN`

df = df.replace('?', np.nan)

In [ ]:
# again view frequency counts of values in categorical variables

for var in categorical:

    print(df[var].value_counts())

# d) Number of labels: cardinality

In [ ]:
# check for cardinality in categorical variables

for var in categorical:

    print(var, 'contains', len(df[var].unique()), 'labels')

# e) Category distribution

In [ ]:
# Calculate and visualise the category distribution for each categorical feature with diagram
categorical_features = df.select_dtypes(include=['object']).columns
for feature in categorical_features:
    plt.figure(figsize=(10, 4))
    sns.countplot(data=df, x=feature)
    plt.title(f'Distribution of {feature}')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# Calculate and visualise the category distribution for each categorical feature with boxplot
categorical_features = df.select_dtypes(include=['object']).columns
for feature in categorical_features:
    plt.figure(figsize=(10, 4))
    sns.boxplot(data=df, x=feature)
    plt.title(f'Distribution of {feature}')
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# sns.pairplot(df, hue='income')
# plt.show()

In [ ]:
# Calculate and visualise the category distribution for 'income' with diagram
for feature in categorical_features:
    plt.figure(figsize=(10, 4))
    sns.countplot(data=df, x=feature, hue='income')
    plt.title(f'{feature} vs Income')
    plt.xticks(rotation=90)
    plt.show()

# f) Numerical variables

In [ ]:
# find numerical variables

numerical = [var for var in df.columns if df[var].dtype!='O']

print('There are {} numerical variables\n'.format(len(numerical)))

print('The numerical variables are :', numerical)

In [ ]:
# view the numerical variables

df[numerical].head()

In [ ]:
# check missing values in numerical variables

df[numerical].isnull().sum()

# Step 3 - Preprocessing

# a) Handling missing value

After changing the '?' above to NaN, now the categorical data will have a null value (missing value). Let's clean up the missing data

In [ ]:
# Check missing values in categorical variables again
df[categorical].isnull().sum()

In [ ]:
# print categorical variables with missing data

for col in categorical:
    if df[col].isnull().mean()>0:
        print(col, (df[col].isnull().mean()))

In [ ]:
# impute missing categorical variables with most frequent value

for df2 in [df]:
    df2['workclass'].fillna(df['workclass'].mode()[0], inplace=True)
    df2['occupation'].fillna(df['occupation'].mode()[0], inplace=True)
    df2['native_country'].fillna(df['native_country'].mode()[0], inplace=True)

In [ ]:
# again check missing values in categorical variables again
df[categorical].isnull().sum()

There's no NaN now. And let's go do the next step ^_^

# b) Encoder categorial variables

In [ ]:
!pip install category_encoders

In [ ]:
# define the columns you want to encode
category_column = ['workclass', 'education', 'marital_status', 'occupation',
                   'relationship','race', 'sex', 'native_country']

1. One-Hot Encoding: Converts each category into a binary column (0 or 1). This is a commonly used method and works well for linear models and many machine learning algorithms.

In [ ]:
# encode remaining variables with One-Hot Encoding
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=category_column)

data1 = encoder.fit_transform(df)

In [ ]:
data1.head()

In [ ]:
data1.shape

After one-hot encoding, there are 106 columns. It seems too much now :)

Let's try another encoding...

2. Binary Encoding: Converts categories into binary representations, and then combines the bits into numerical features. This reduces the number of columns created compared to one-hot encoding.

In [ ]:
# encode remaining variables with Binary Encoding
from category_encoders import BinaryEncoder

binary_encoder = BinaryEncoder(cols=category_column)
data2 = binary_encoder.fit_transform(df)

In [ ]:
data2.head()

In [ ]:
data2.shape

After one-hot encoding, there are 37 columns. It is still too much :)

Let's try another encoding...

3. Label Encoder: Label encoding is a common technique used in machine learning to convert categorical variables into numerical representations. It is particularly useful when working with algorithms that cannot directly handle categorical data. However, when dealing with datasets that have multiple columns containing categorical variables, applying label encoding individually to each column can be time-consuming and error-prone. In such cases, a multi-column label encoding approach can be employed to streamline the process.

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

# Initialitation LabelEncoder
label_encoder = preprocessing.LabelEncoder()

# create a copy of data3 to df
data3 = df.copy()

data3 = data3.apply(lambda col: label_encoder.fit_transform(col) if col.dtype == 'object' else col)

In [ ]:
data3.head()

In [ ]:
data3.shape

In [ ]:
data3.info()

Now we have a numeric dataset with 15 columns after encoder. We will use this dataset (data3) to build the model classifier.

# c) Correlation

In [ ]:
# Korelasi setiap fitur
correlations = data3.corr()
print(correlations)

In [ ]:
sns.heatmap(data3.corr(), annot=True, cmap='RdYlGn', linewidths=0.5)
# data.corr() ---> correlation matrix
fig = plt.gcf()
fig.set_size_inches(20,20)
plt.show()

# Step 4 -Split data into separate training and test set

In [ ]:
# Split data for training and test set

X = data3.drop(['income'], axis=1)

y = data3['income']

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# Seleksi fitur menggunakan Chi-squared test
selector = SelectKBest(chi2, k=2)
X_new = selector.fit_transform(X, y)

In [ ]:
# split X and y into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

K-Fold Cross Validation

In [ ]:
# from sklearn.model_selection import KFold

# # Membuat objek KFold dengan 5 lipatan
# kf = KFold(n_splits=5)

# for train_index, test_index in kf.split(X):
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Step 5 - Feature Engineering

In [ ]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

# a) Reduksi Dimensi: Latent Discriminant  Analysis (LDA)

Latent Dirichlet Allocation is a probabilistic model used for topic modeling.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

#Fit the LDA model
model = LinearDiscriminantAnalysis()
model.fit(X, y)

In [ ]:
# Chek class/category balance
print(df['income'].value_counts())

The target class shows an imbalance in the income data of people <=50k and >50k. The gap in the number of classes is around 3:1. For cases where the classification of these classes is very unbalanced, it is necessary to resample the data.

# b) Resampling
Oversampling on Minority Classes: Adding copies of the minority class examples to balance the dataset. A frequently used technique is SMOTE (Synthetic Minority Over-sampling Technique). Undersampling the Majority Class: Reduces the number of examples from the majority class to balance the dataset.

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

# Oversampling
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

# Undersampling on the oversampled data
undersampler = RandomUnderSampler()
X_resampled, y_resampled = undersampler.fit_resample(X_resampled, y_resampled)

# Combining oversampling and undersampling on the undersampled data
smote_enn = SMOTEENN()
X_resampled, y_resampled = smote_enn.fit_resample(X_resampled, y_resampled)

In [ ]:
# print the count of each class in the resampled data
print(y_resampled.value_counts())

Although the number of samples between these two classes is not exactly the same, the difference is not very significant. Therefore, this data can be considered fairly balanced.

# Step 6 - Feature Scaling
We use RobustScaler from sklearn.preprocessing to perform feature scaling. This scaling uses a method that is more robust to outliers than standard scaling. RobustScaler removes the median and scales the data by quartiles (IQR: Q3 - Q1). This transformation transforms each feature into the same range and is important for some machine learning algorithms.

In [ ]:
X_train.head()

In [ ]:
cols = X_train.columns

In [ ]:
# Feature scalling

from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

# Transformasi Data
X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [ ]:
# Konvertion to DataFrame

X_train = pd.DataFrame(X_train, columns=[cols])

In [ ]:
# Konvertion to DataFrame

X_test = pd.DataFrame(X_test, columns=[cols])

In [ ]:
X_train.head()

We now have X_train dataset ready to be fed into the Gaussian Naive Bayes classifier. Let's do it as follows.

# Step 7 - Modelling Backpropogation to predict the adult

# 1 Menggunakan splitting data

Now in the next step, we have to start initializing the hyperparameters. We will input the learning rate, iterations, input size, number of hidden layers, and number of output layers.

In [ ]:
# Sigmoid activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Mean squared error function
def mean_squared_error(y_pred, y_true):
    return np.mean((y_pred - y_true) ** 2)

# Function to calculate accuracy
def accuracy(y_true, y_pred):
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_true, axis=1)
    return np.mean(y_pred_classes == y_true_classes)

In [ ]:
# Model parameter
output_size = 2  # class label
input_size = X_train.shape[1]  # The number of features
hidden_size = 64  # Hidden layer size
learning_rate = 0.1
iterations = 5000
N = X_train.shape[0]  # The number of sampel

In [ ]:
# Weights initiatilation (W)
np.random.seed(10)
W1 = np.random.randn(input_size, hidden_size)
W2 = np.random.randn(hidden_size, output_size)

# One-hot encode y_train and y_test
y_train_one_hot = np.eye(output_size)[y_train]
y_test_one_hot = np.eye(output_size)[y_test]

# DataFrame to save results of model traning
results = pd.DataFrame(columns=["mse", "accuracy"])

# List to store the prediction result and the actual label
# prediction = []

In [ ]:
# The training loop
for itr in range(iterations):
    # Feedforward propagation
    Z1 = np.dot(X_train, W1)
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2)
    A2 = sigmoid(Z2)

    # Calculate error
    mse = mean_squared_error(A2, y_train_one_hot)
    acc = accuracy(y_train_one_hot, A2)
    new_row = pd.DataFrame({"mse": [mse], "accuracy": [acc]})
    results = pd.concat([results, new_row], ignore_index=True)

    # Backpropagation
    E1 = A2 - y_train_one_hot
    dW1 = E1 * A2 * (1 - A2)
    E2 = np.dot(dW1, W2.T)
    dW2 = E2 * A1 * (1 - A1)

    # Update weights
    W2_update = np.dot(A1.T, dW1) / N
    W1_update = np.dot(X_train.T, dW2) / N
    W2 = W2 - learning_rate * W2_update
    W1 = W1 - learning_rate * W1_update

In [ ]:
# Display training results
print(results)

In [ ]:
# Visualizing the results

results.mse.plot(title="Mean Squared Error")
plt.show()

results.accuracy.plot(title="Accuracy")
plt.show()

In [ ]:
# Test the model
Z1 = np.dot(X_test, W1)
A1 = sigmoid(Z1)
Z2 = np.dot(A1, W2)
A2 = sigmoid(Z2)
test_acc = accuracy(np.eye(output_size)[y_test], A2)
print("Test accuracy: {}".format(test_acc))

# Model Evaluation

In [ ]:
# Evaluation on test data
Z1_test = np.dot(X_test, W1)
A1_test = sigmoid(Z1_test)
Z2_test = np.dot(A1_test, W2)
A2_test = sigmoid(Z2_test)

In [ ]:
mse_test = mean_squared_error(A2_test, y_test_one_hot)
acc_test = accuracy(y_test_one_hot, A2_test)

# Calculating other evaluation metrics
y_pred_classes = np.argmax(A2_test, axis=1)
conf_matrix = confusion_matrix(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='macro')
recall = recall_score(y_test, y_pred_classes, average='macro')
f1 = f1_score(y_test, y_pred_classes, average='macro')

print(f"Test MSE: {mse_test}")
print(f"Test Accuracy: {acc_test}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

In [ ]:
!pip install seaborn

In [ ]:
# Viewing confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# 2. Menggunakan K-Fold Cross Validation

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
# Sigmoid activation function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Mean squared error function
def mean_squared_error(y_pred, y_true):
    return np.mean((y_pred - y_true) ** 2)

# Function to calculate accuracy
def accuracy(y_true, y_pred):
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_true, axis=1)
    return np.mean(y_pred_classes == y_true_classes)

In [ ]:
# Model hyperparamter
output_size = 2  # Count of class
hidden_size = 64  # Hidden layer size (neuron)
learning_rate = 0.01
iterations = 1000
k_folds = 5  # Count of fold for k-fold cross-validation

In [ ]:
# k-fold initialitation
kf = KFold(n_splits=k_folds)

# DataFrame to save results of model traning
results = pd.DataFrame(columns=["fold", "mse", "accuracy"])

In [ ]:
# Loop k-fold cross-validation
fold_idx = 1

# Make objek KFold with 3 cv
kf = KFold(n_splits=3)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # One-hot encode y_train and y_test
    y_train_one_hot = np.eye(output_size)[y_train]
    y_test_one_hot = np.eye(output_size)[y_test]

    # Weights initialitation
    np.random.seed(42)
    W1 = np.random.randn(X_train.shape[1], hidden_size)
    W2 = np.random.randn(hidden_size, output_size)

    # Loop pelatihan
    for itr in range(iterations):
        # Feedforward propagation
        Z1 = np.dot(X_train, W1)
        A1 = sigmoid(Z1)
        Z2 = np.dot(A1, W2)
        A2 = sigmoid(Z2)

        # Calculate error
        mse = mean_squared_error(A2, y_train_one_hot)
        acc = accuracy(y_train_one_hot, A2)

        # Backpropagation
        E1 = A2 - y_train_one_hot
        dW1 = E1 * A2 * (1 - A2)
        E2 = np.dot(dW1, W2.T)
        dW2 = E2 * A1 * (1 - A1)

        # Update weights
        W2_update = np.dot(A1.T, dW1) / X_train.shape[0]
        W1_update = np.dot(X_train.T, dW2) / X_train.shape[0]
        W2 = W2 - learning_rate * W2_update
        W1 = W1 - learning_rate * W1_update

    # Evaluation on test data
    Z1_test = np.dot(X_test, W1)
    A1_test = sigmoid(Z1_test)
    Z2_test = np.dot(A1_test, W2)
    A2_test = sigmoid(Z2_test)

    mse_test = mean_squared_error(A2_test, y_test_one_hot)
    acc_test = accuracy(y_test_one_hot, A2_test)

    new_row = pd.DataFrame({"fold": [fold_idx], "mse": [mse_test], "accuracy": [acc_test]})
    results = pd.concat([results, new_row], ignore_index=True)

    fold_idx += 1

In [ ]:
# Display training results of all cv
print(results)